# 6. Optimizing Model Parameters

## Pre-requisite Code

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

## Hyperparameters

In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## Optimization Loop

하나의 epoch는 다음 두 부분으로 구성된다.
- train loop - 학습용 dataset을 iterate하고 최적의 parameter로 수렴한다.
- validation/test loop - model 성능이 개선되고 있는지 확인하기 위해 test datset을 iterate한다.

## Loss Function

> 일반적인 loss function에는,  
regression task에 사용하는 `nn.MSELoss`,  
classification에 사용하는 `nn.NLLLoss`,  
그리고 `nn.LogSoftmax`와 `nn.NLLLoss`를 합친 `nn.CrossEntropyLoss` 등이 있다.

> model의 output logit을 `nn.CrossEntropyLoss`에 전달하여 logit을 정규화하고 예측 오류를 계산한다.

In [3]:
# loss function initializing
loss_fn = nn.CrossEntropyLoss()

## Optimizer

> 모든 최적화 절차는 `optimizer` 개체에 캡슐화된다.  

In [4]:
# Optimizer initializing
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

train loop에서 최적화는 세 단계로 이루어진다.
- `optimizer.zero_grad()`를 호출하여 model parameter의 gradient를 재설정한다.  
기본적으로 gradient는 더해지기 때문에 중복 계산을 막기 위해 반복할 때마다 명시적으로 0으로 설정한다.
- `loss.backwards()`를 호출하여 prediction loss를 backpropagate한다.  
pytorch는 각 parameter에 대해 loss의 gradient를 저장한다.
- gradient를 계산한 뒤, `optimizer.step()`을 호출하여 backpropagate 단계에서 수집된 gradient로 parameter를 조정한다.

## Full Implemetation

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # prediction과 loss 계산
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f'loss: {loss:>7f} [{current:>5d}/{size:>5d}]')

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /= size
    print(f'Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n')

> loss function과 optimizer를 초기화하고, `train_loop`와 `test_loop`에 전달한다.  
model의 성능 향상을 알아보기 위해 epoch수를 자유롭게 증가시켜볼 수 있다.

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f'Epoch {t+1}\n---------------------------')
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(train_dataloader, model, loss_fn)
print('Done!')

Epoch 1
---------------------------
loss: 2.302263 [   64/60000]
loss: 2.291017 [ 6464/60000]
loss: 2.274541 [12864/60000]
loss: 2.272569 [19264/60000]
loss: 2.242208 [25664/60000]
loss: 2.220980 [32064/60000]
loss: 2.225939 [38464/60000]
loss: 2.191874 [44864/60000]
loss: 2.192827 [51264/60000]
loss: 2.153119 [57664/60000]
Test Error: 
 Accuracy: 42.2%, Avg loss: 2.154046 

Epoch 2
---------------------------
loss: 2.167067 [   64/60000]
loss: 2.156435 [ 6464/60000]
loss: 2.098273 [12864/60000]
loss: 2.117744 [19264/60000]
loss: 2.049263 [25664/60000]
loss: 1.998094 [32064/60000]
loss: 2.029153 [38464/60000]
loss: 1.946053 [44864/60000]
loss: 1.957492 [51264/60000]
loss: 1.877311 [57664/60000]
Test Error: 
 Accuracy: 52.5%, Avg loss: 1.875097 

Epoch 3
---------------------------
loss: 1.917251 [   64/60000]
loss: 1.883585 [ 6464/60000]
loss: 1.761492 [12864/60000]
loss: 1.805805 [19264/60000]
loss: 1.683458 [25664/60000]
loss: 1.642998 [32064/60000]
loss: 1.676001 [38464/60000]
loss: